## 영문텍스트에 anki단어장 만들기

실습 목표
영어 텍스트를 입력하면, 중요한 어휘를 추출해 Anki 카드 형식의 단어장 데이터를 생성하는 프롬프트를 설계한다.

프롬프트 내에 시스템 지시사항, 예시, 출력 형식을 포함한다.

### 문제
다음 요구사항을 만족하는 LLM 프롬프트를 작성하시오.

### 요구사항

### 시스템 지시사항

LLM이 영어 텍스트에서 학습 가치가 있는 주요 단어 또는 어구를 추출하여 단어장으로 만들어야 한다.

너무 쉬운 단어는 제외하고, 고유명사, 전문용어, 관용구 등도 포함한다.

각 카드에는 (1)단어/어구, (2)뜻(영어 또는 한글), (3)품사, (4)예문(영어)을 포함한다.

각 카드는 Anki에서 바로 쓸 수 있는 포맷(각 항목은 세미콜론 ; 으로 구분, 한 줄에 한 카드)으로 출력한다.

### 예시

- 입력 텍스트 예시:

The committee will convene next week to discuss the new policy proposals and reach a consensus.

- 출력 예시:

convene;to come together for a meeting;verb;The committee will convene next week.
consensus;general agreement;noun;The group reached a consensus after a long discussion.
proposal;a suggestion or plan;noun;He submitted a proposal for the new project.

- LLM 출력 형식

각 카드: 단어 또는 어구;뜻;품사;영어 예문
모든 카드는 한 줄에 한 개, 세미콜론(;)으로 필드 구분
불필요한 부가설명, 줄바꿈, 문장 없음

- DataFrame 시각화

LLM 응답을 파싱해서 다음과 같이 DataFrame으로 출력하세요.
<img src="https://private-user-images.githubusercontent.com/39724768/458282429-399dc257-7fc1-475a-9ffb-3cb408aa73b3.png?jwt=eyJhbGciOiJIUzI1NiIsInR5cCI6IkpXVCJ9.eyJpc3MiOiJnaXRodWIuY29tIiwiYXVkIjoicmF3LmdpdGh1YnVzZXJjb250ZW50LmNvbSIsImtleSI6ImtleTUiLCJleHAiOjE3NTA4MDcxMzAsIm5iZiI6MTc1MDgwNjgzMCwicGF0aCI6Ii8zOTcyNDc2OC80NTgyODI0MjktMzk5ZGMyNTctN2ZjMS00NzVhLTlmZmItM2NiNDA4YWE3M2IzLnBuZz9YLUFtei1BbGdvcml0aG09QVdTNC1ITUFDLVNIQTI1NiZYLUFtei1DcmVkZW50aWFsPUFLSUFWQ09EWUxTQTUzUFFLNFpBJTJGMjAyNTA2MjQlMkZ1cy1lYXN0LTElMkZzMyUyRmF3czRfcmVxdWVzdCZYLUFtei1EYXRlPTIwMjUwNjI0VDIzMTM1MFomWC1BbXotRXhwaXJlcz0zMDAmWC1BbXotU2lnbmF0dXJlPTg1ODU5NTRlZTI0OGMyNzE2N2ZiNTk2NTIxZTg2ZDFhZTQwYTI5M2M2NTczZjlmOWNiNmY2Y2I0NmJjZmM4NmImWC1BbXotU2lnbmVkSGVhZGVycz1ob3N0In0.QWrB9Wc-cKnnPWuKKQvdwf_tqMGFh8MaRoqzIIxzBDM">

In [ ]:
# 필요한 라이브러리 임포트
from openai import OpenAI
from google.colab import userdata
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
# --- 1. 우리가 만든 시스템 메시지 (프롬프트) ---
# 역할 부여
system_message_for_anki = """
당신은 영어 텍스트에서 학습 가치가 있는 주요 단어와 어구를
추출하여 Anki 단어장 형식으로 만드는 AI 어시스턴트입니다.
"""
# 기존 system_message_for_anki 내용에 이어서, 단어 선정 기준에 대한 설명을 추가.
system_message_for_anki += "\n\n### 단어/어구 선정 기준:"
system_message_for_anki += "\n- 너무 쉬운 단어 (예: a, the, is, have)는 제외하고, 학습자에게 유용하다고 판단되는 단어를 선택해주세요."
system_message_for_anki += "\n- 고유명사, 전문용어, 또는 관용적인 표현도 중요한 어휘로 간주하여 포함해주세요."
# 기존 system_message_for_anki 내용에 이어서, 카드 구성 항목에 대한 설명을 추가.
system_message_for_anki += "\n\n### 카드 구성 항목:"
system_message_for_anki += "\n각 카드에는 다음 네 가지 정보를 포함해야 합니다:"
system_message_for_anki += "\n1. 단어/어구: 추출된 단어나 어구"
system_message_for_anki += "\n2. 뜻: 해당 단어/어구의 한국어"
system_message_for_anki += "\n3. 품사: 해당 단어/어구의 품사 (예: noun, verb, adjective)"
system_message_for_anki += "\n4. 예문: 해당 단어/어구가 사용된 원본 영어 텍스트 내의 예문"
# 기존 system_message_for_anki 내용에 이어서, 출력 형식에 대한 설명을 추가.
system_message_for_anki += "\n\n### 출력 형식 (Anki 포맷):"
system_message_for_anki += "\n- 각 카드는 한 줄로 구성되며, 각 항목은 세미콜론(;)으로 구분합니다."
system_message_for_anki += "\n- 형식: 단어/어구;뜻;품사;예문"
system_message_for_anki += "\n- 예시: convene;to come together for a meeting;verb;The committee will convene next week."
system_message_for_anki += "\n- 중요: 출력 시 부가적인 설명, 인사말, 번호 매기기 등은 절대 포함하지 말고, 오직 요청된 카드 데이터만 한 줄씩 연속으로 출력해야 합니다."
# 기존 system_message_for_anki 내용에 이어서, 입출력 예시를 추가.
system_message_for_anki += "\n\n### 예시:"
system_message_for_anki += "\n입력 텍스트 예시:"
system_message_for_anki += "\nThe committee will convene next week to discuss the new policy proposals and reach a consensus."
system_message_for_anki += "\n출력 예시 (Anki 카드 형식):"
system_message_for_anki += "\nconvene;to come together for a meeting;verb;The committee will convene next week."
system_message_for_anki += "\nconsensus;general agreement;noun;The group reached a consensus after a long discussion."
system_message_for_anki += "\nproposal;a suggestion or plan;noun;He submitted a proposal for the new project."


# --- 2. Anki 단어장 생성 함수 정의 ---
def generate_anki_cards(input_text, system_prompt, model_name="gpt-4.1-nano", api_key_value=None, temp=0.2, max_output_tokens=1024):
    """
    주어진 영어 텍스트와 시스템 프롬프트를 사용하여 Anki 단어장 형식의 데이터를 생성합니다.

    Args:
        input_text (str): 단어장을 생성할 원본 영어 텍스트.
        system_prompt (str): LLM에게 전달할 시스템 메시지 (역할, 지시사항, 예시 등 포함).
        model_name (str, optional): 사용할 OpenAI 모델 이름. 기본값 'gpt-4o-mini'.
        api_key_value (str, optional): 사용할 OpenAI API 키. None이면 오류 발생.
        temp (float, optional): 생성 결과의 무작위성 조절 (0.0 ~ 2.0). 낮을수록 결정적. 기본값 0.2.
        max_output_tokens (int, optional): 생성될 최대 토큰 수. 기본값 1024.

    Returns:
        str: Anki 카드 형식으로 생성된 단어장 데이터. 오류 발생 시 None 반환.
    """
    # API 키가 제공되었는지 확인
    if not api_key_value:
        print("오류: OpenAI API 키가 제공되지 않았습니다.")
        return None

    try:
        # OpenAI API 클라이언트 초기화 시 API 키 직접 전달
        client = OpenAI(api_key=OPENAI_API_KEY)

        # LLM에게 전달할 사용자 메시지 구성
        user_prompt_for_anki = f"""다음 영어 텍스트에서 주요 어휘를 추출하여 Anki 단어장을 만들어주세요.

영어 텍스트:
{input_text}
"""

        # OpenAI API에 채팅 완성(Chat Completion) 요청
        chat_completion_response = client.chat.completions.create(
            model=model_name,  # 사용할 모델 지정
            messages=[
                {
                    "role": "system",
                    "content": system_prompt  # 우리가 상세히 작성한 시스템 프롬프트 전달
                },
                {
                    "role": "user",
                    "content": user_prompt_for_anki # 사용자가 입력한 텍스트 전달
                }
            ],
            temperature=temp,  # 결과의 다양성 조절
            max_tokens=max_output_tokens,  # 최대 생성 토큰 수 제한
            top_p=1,  # 뉴클리어스 샘플링 (1로 설정하면 특별히 사용 안 함)
            frequency_penalty=0,  # 특정 단어 반복 사용 억제 정도
            presence_penalty=0  # 새로운 주제 도입 장려 정도
        )
        # 생성된 응답 텍스트를 반환 (앞뒤 공백 제거)
        return chat_completion_response.choices[0].message.content.strip()

    except Exception as e:
        # API 호출 중 오류 발생 시 메시지 출력 및 None 반환
        print(f"Anki 단어장 생성 중 오류 발생: {e}")
        return None

# --- 3. 함수 사용 및 결과 확인 예시 ---
if __name__ == '__main__':
    # 코랩 비밀에서 API 키 불러오기
    # '비밀' 탭에 'OPENAI_API_KEY'라는 이름으로 API 키가 저장되어 있어야 함
    my_openai_api_key = userdata.get('OPENAI_API_KEY')

    if not my_openai_api_key:
        print("코랩 비밀에서 'OPENAI_API_KEY'를 찾을 수 없습니다. API 키를 설정해주세요.")
    else:
        # --- 여기에 우리가 함께 만든 system_message_for_anki 변수가 정의되어 있어야 합니다 ---
        # 예시로 간단히 다시 정의 (실제로는 이전 단계에서 만든 긴 프롬프트를 사용해야 함)
        system_message_for_anki_example = """당신은 영어 텍스트에서 학습 가치가 있는 주요 단어와 어구를 추출하여 Anki 단어장 형식으로 만드는 AI 어시스턴트입니다.

### 단어/어구 선정 기준:
- 너무 쉬운 단어 (예: a, the, is, have)는 제외하고, 학습자에게 유용하다고 판단되는 단어를 선택해주세요.
- 고유명사, 전문용어, 또는 관용적인 표현도 중요한 어휘로 간주하여 포함해주세요.

### 카드 구성 항목:
각 카드에는 다음 네 가지 정보를 포함해야 합니다:
1. 단어/어구: 추출된 단어나 어구
2. 뜻: 해당 단어/어구의 한국어 의미
3. 품사: 해당 단어/어구의 품사 (예: noun, verb, adjective)
4. 예문: 해당 단어/어구가 사용된 원본 영어 텍스트 내의 예문

### 출력 형식 (Anki 포맷):
- 각 카드는 한 줄로 구성되며, 각 항목은 세미콜론(;)으로 구분합니다.
- 형식: 단어/어구;뜻;품사;예문
- 예시: convene;to come together for a meeting;verb;The committee will convene next week.
- 중요: 출력 시 부가적인 설명, 인사말, 번호 매기기 등은 절대 포함하지 말고, 오직 요청된 카드 데이터만 한 줄씩 연속으로 출력해야 합니다.

### 예시:
입력 텍스트 예시:
The committee will convene next week to discuss the new policy proposals and reach a consensus.
출력 예시 (Anki 카드 형식):
convene;to come together for a meeting;verb;The committee will convene next week.
consensus;general agreement;noun;The group reached a consensus after a long discussion.
proposal;a suggestion or plan;noun;He submitted a proposal for the new project."""
        # ---------------------------------------------------------------------------------

        print("--- 시스템 프롬프트 (요약본 예시) ---")
        print(system_message_for_anki_example) # 실제로는 이전에 만든 긴 system_message_for_anki를 사용해야 함
        print("\n" + "="*50 + "\n")

        # 테스트할 영어 텍스트
        test_english_input = """
        The committee will convene next week to discuss the new policy proposals and reach a consensus.
        Artificial intelligence is rapidly transforming various industries, leading to unprecedented advancements and ethical considerations.
        Many startups are leveraging cutting-edge technology to disrupt traditional markets.
        """
        model_to_use = "gpt-4.1-nano"

        print(f"--- '{model_to_use}' 모델로 Anki 단어장 생성 요청 ---")
        model_to_use = "gpt-4o-mini"
        anki_cards_output = generate_anki_cards(
            input_text=test_english_input,
            system_prompt=system_message_for_anki_example, # 실제로는 긴 system_message_for_anki 사용
            model_name=model_to_use,
            api_key_value=my_openai_api_key,
            temp=0.3 # 온도를 살짝 조절하여 일관성 유지
        )

        if anki_cards_output:
            print("\n--- 생성된 Anki 단어장 (Raw Text) ---")
            print(anki_cards_output)

            # (선택 사항) 생성된 텍스트를 DataFrame으로 파싱하여 보기 좋게 출력
            print("\n--- 생성된 Anki 단어장 (DataFrame) ---")
            import pandas as pd
            lines_for_df = [line.strip() for line in anki_cards_output.strip().split('\n') if line.strip()]
            data_list_for_df = []
            for line_item in lines_for_df:
                parts_of_line = line_item.split(';', 3)
                if len(parts_of_line) == 4:
                    data_list_for_df.append(parts_of_line)
                else:
                    print(f"경고: 다음 라인은 필드 개수({len(parts_of_line)})가 맞지 않아 DataFrame에서 제외됩니다: {line_item}")

            if data_list_for_df:
                anki_dataframe = pd.DataFrame(data_list_for_df, columns=['Word/Phrase', 'Definition', 'PoS', 'Example Sentence'])
                try:
                    from IPython.display import display
                    display(anki_dataframe)
                except ImportError:
                    print(anki_dataframe)
            else:
                print("파싱할 수 있는 유효한 Anki 카드 데이터가 없습니다.")
        else:
            print("\n단어장 생성에 실패했습니다.")